# 1. 윤석열

In [3]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "윤석열_조선일보_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 


# 기사 내용을 추출하는 함수
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        # 다양한 본문 태그 탐지 및 텍스트 추출
        content = ""

        # <div itemprop="articleBody">
        article_body = soup.find("div", itemprop="articleBody")
        if article_body:
            paragraphs = article_body.find_all(["p", "div"], recursive=False)
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div itemprop="articleBody" id="id_articleBody">
        if not content.strip():
            article_body_id = soup.find("div", itemprop="articleBody", id="id_articleBody")
            if article_body_id:
                paragraphs = article_body_id.find_all(["p", "div"], recursive=False)
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="article-body">
        if not content.strip():
            article_body_div = soup.find("div", class_="article-body")
            if article_body_div:
                paragraphs = article_body_div.find_all("p")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="cont_body">
        if not content.strip():
            cont_body = soup.find("div", class_="cont_body")
            if cont_body:
                paragraphs = cont_body.find_all("div")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <p class="article-body__content">
        if not content.strip():
            article_paragraphs = soup.find_all("p", class_="article-body__content")
            content = "\n".join([p.get_text(strip=True) for p in article_paragraphs])

        # <div id="content">
        if not content.strip():
            content_div = soup.find("div", id="content")
            if content_div:
                paragraphs = content_div.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="content">
        if not content.strip():
            content_class = soup.find("div", class_="content")
            if content_class:
                paragraphs = content_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="articleBody">
        if not content.strip():
            article_body_class = soup.find("div", class_="articleBody")
            if article_body_class:
                paragraphs = article_body_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <font> 태그 + <br> 처리
        if not content.strip():
            font_tag = soup.find("font")
            if font_tag:
                # <br> 기준으로 텍스트를 합침
                content = "\n".join([text.strip() for text in font_tag.stripped_strings])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없을 경우 None 반환
        return content if content else None

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None



# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
for idx, row in df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Chosun_YSY_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")

    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (1/200): http://monthly.chosun.com/client/mdaily/daily_view.asp?idx=14722&NewsNumb=20220214722
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200\Chosun_YSY_1_200.txt
크롤링 중 (2/200): https://biz.chosun.com/policy/politics/election2022/2022/03/02/BKLNEDYNRRD4DLXZFSQK74GWBA/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200\Chosun_YSY_2_200.txt
크롤링 중 (3/200): https://biz.chosun.com/policy/politics/election2022/2022/02/17/XZNRTZ3EHRDY7GU6U2KLF42TZM/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200\Chosun_YSY_3_200.txt
크롤링 중 (4/200): https://biz.chosun.com/policy/politics/election2022/2022/02/16/RNCRBZAS45BALITF2OE4MKFOHM/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200\Chosun_YSY_4_200.txt
크롤링 중 (5/200): https://www.chosun.com/politics/election2022/2022/02/25/PNEGNQDREJDONH5JMMNXPD6BWU/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200\Chosun_YSY_5_200.txt
크롤링 중 (6/200): http://monthly.chosun.com/client/mdaily/daily_view.asp?idx=14801&NewsNumb=20220314

In [14]:
# 빠뜨린 기사들 스크랩
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "윤석열_조선일보_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 


# 기사 내용을 추출하는 함수
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        # 다양한 본문 태그 탐지 및 텍스트 추출
        content = ""

        # <div itemprop="articleBody">
        article_body = soup.find("div", itemprop="articleBody")
        if article_body:
            paragraphs = article_body.find_all(["p", "div"], recursive=False)
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div itemprop="articleBody" id="id_articleBody">
        if not content.strip():
            article_body_id = soup.find("div", itemprop="articleBody", id="id_articleBody")
            if article_body_id:
                paragraphs = article_body_id.find_all(["p", "div"], recursive=False)
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="article-body">
        if not content.strip():
            article_body_div = soup.find("div", class_="article-body")
            if article_body_div:
                paragraphs = article_body_div.find_all("p")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="cont_body">
        if not content.strip():
            cont_body = soup.find("div", class_="cont_body")
            if cont_body:
                paragraphs = cont_body.find_all("div")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <p class="article-body__content">
        if not content.strip():
            article_paragraphs = soup.find_all("p", class_="article-body__content")
            content = "\n".join([p.get_text(strip=True) for p in article_paragraphs])

        # <div id="content">
        if not content.strip():
            content_div = soup.find("div", id="content")
            if content_div:
                paragraphs = content_div.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="content">
        if not content.strip():
            content_class = soup.find("div", class_="content")
            if content_class:
                paragraphs = content_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="articleBody">
        if not content.strip():
            article_body_class = soup.find("div", class_="articleBody")
            if article_body_class:
                paragraphs = article_body_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <font> 태그 + <br> 처리
        if not content.strip():
            font_tag = soup.find("font")
            if font_tag:
                # <br> 기준으로 텍스트를 합침
                content = "\n".join([text.strip() for text in font_tag.stripped_strings])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없을 경우 None 반환
        return content if content else None

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
# 크롤링할 특정 행 번호 리스트 (1-based index)
selected_rows = [59]

# 0-based 인덱스로 변환 (Python 인덱스는 0부터 시작)
selected_rows = [row - 1 for row in selected_rows]

# 데이터프레임에서 해당 행 번호만 필터링
filtered_df = df.iloc[selected_rows]

# 크롤링 및 저장
for idx, row in filtered_df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Chosun_YSY_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")


    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (59/200): http://monthly.chosun.com/client/news/viw.asp?ctcd=A&nNewsNumb=202203100016
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\윤석열 기사 200\Chosun_YSY_59_200.txt


# 2. 이재명

In [5]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "이재명_조선일보_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\조선일보\이재명 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정


# 기사 본문 추출 함수
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        # 다양한 본문 태그 탐지 및 텍스트 추출
        content = ""

        # <div itemprop="articleBody">
        article_body = soup.find("div", itemprop="articleBody")
        if article_body:
            paragraphs = article_body.find_all(["p", "div"], recursive=False)
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div itemprop="articleBody" id="id_articleBody">
        if not content.strip():
            article_body_id = soup.find("div", itemprop="articleBody", id="id_articleBody")
            if article_body_id:
                paragraphs = article_body_id.find_all(["p", "div"], recursive=False)
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="article-body">
        if not content.strip():
            article_body_div = soup.find("div", class_="article-body")
            if article_body_div:
                paragraphs = article_body_div.find_all("p")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="cont_body">
        if not content.strip():
            cont_body = soup.find("div", class_="cont_body")
            if cont_body:
                paragraphs = cont_body.find_all("div")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <p class="article-body__content">
        if not content.strip():
            article_paragraphs = soup.find_all("p", class_="article-body__content")
            content = "\n".join([p.get_text(strip=True) for p in article_paragraphs])

        # <div id="content">
        if not content.strip():
            content_div = soup.find("div", id="content")
            if content_div:
                paragraphs = content_div.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="content">
        if not content.strip():
            content_class = soup.find("div", class_="content")
            if content_class:
                paragraphs = content_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="articleBody">
        if not content.strip():
            article_body_class = soup.find("div", class_="articleBody")
            if article_body_class:
                paragraphs = article_body_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <font> 태그 + <br> 처리
        if not content.strip():
            font_tag = soup.find("font")
            if font_tag:
                # <br> 기준으로 텍스트를 합침
                content = "\n".join([text.strip() for text in font_tag.stripped_strings])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없을 경우 None 반환
        return content if content else None

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None


# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
for idx, row in df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Chosun_LJM_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")

    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (1/200): https://biz.chosun.com/policy/politics/election2022/2022/02/21/WXX5XKDCL5DWHGH46QJKM63W7Y/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_1_200.txt
크롤링 중 (2/200): http://monthly.chosun.com/client/mdaily/daily_view.asp?idx=14670&NewsNumb=20220214670
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_2_200.txt
크롤링 중 (3/200): https://biz.chosun.com/policy/politics/election2022/2022/02/15/MXRQ2JRW5ZABNAHAA6JKMU3TOE/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_3_200.txt
크롤링 중 (4/200): http://monthly.chosun.com/client/mdaily/daily_view.asp?idx=14696&NewsNumb=20220214696
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_4_200.txt
크롤링 중 (5/200): https://biz.chosun.com/policy/politics/election2022/2022/02/17/KQGKQFL7ZNHRBLMJE3FAPKAIQA/
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_5_200.txt
크롤링 중 (6/200): https://biz.chosun.com/policy/politics/election2022/2022/02/19/OAXFDEWM6NAPZAILI

In [13]:
# 빠뜨린 기사 추출
# 빠뜨린 기사들 스크랩
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화
chrome_options.add_argument("--headless")  # 셀레늄 창 보이지 않기

# CSV 파일 경로
csv_file = "이재명_조선일보_기사목록200.csv"

# 저장 디렉토리 설정
output_dir = r"C:\Users\Administrator\2024-2\조선일보\이재명 기사 200"
os.makedirs(output_dir, exist_ok=True)

# Selenium WebDriver 설정
s = Service(r'C:\Users\Administrator\2024-2\chromedriver-win64\chromedriver.exe')  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
driver.set_page_load_timeout(60)  # 페이지 로드 타임아웃 설정 


# 기사 내용을 추출하는 함수
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def scrape_article_content_selenium(url):
    try:
        # URL 접속
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기

        # 페이지 소스 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "lxml")

        # 다양한 본문 태그 탐지 및 텍스트 추출
        content = ""

        # <div itemprop="articleBody">
        article_body = soup.find("div", itemprop="articleBody")
        if article_body:
            paragraphs = article_body.find_all(["p", "div"], recursive=False)
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div itemprop="articleBody" id="id_articleBody">
        if not content.strip():
            article_body_id = soup.find("div", itemprop="articleBody", id="id_articleBody")
            if article_body_id:
                paragraphs = article_body_id.find_all(["p", "div"], recursive=False)
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="article-body">
        if not content.strip():
            article_body_div = soup.find("div", class_="article-body")
            if article_body_div:
                paragraphs = article_body_div.find_all("p")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="cont_body">
        if not content.strip():
            cont_body = soup.find("div", class_="cont_body")
            if cont_body:
                paragraphs = cont_body.find_all("div")
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <p class="article-body__content">
        if not content.strip():
            article_paragraphs = soup.find_all("p", class_="article-body__content")
            content = "\n".join([p.get_text(strip=True) for p in article_paragraphs])

        # <div id="content">
        if not content.strip():
            content_div = soup.find("div", id="content")
            if content_div:
                paragraphs = content_div.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="content">
        if not content.strip():
            content_class = soup.find("div", class_="content")
            if content_class:
                paragraphs = content_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <div class="articleBody">
        if not content.strip():
            article_body_class = soup.find("div", class_="articleBody")
            if article_body_class:
                paragraphs = article_body_class.find_all(["p", "div"])
                content = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # <font> 태그 + <br> 처리
        if not content.strip():
            font_tag = soup.find("font")
            if font_tag:
                # <br> 기준으로 텍스트를 합침
                content = "\n".join([text.strip() for text in font_tag.stripped_strings])

        # 불필요한 공백 제거
        content = content.strip()

        # 본문이 없을 경우 None 반환
        return content if content else None

    except Exception as e:
        print(f"크롤링 실패: {url}, 오류: {e}")
        return None



# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 기사 크롤링 및 텍스트 파일 저장
# 크롤링할 특정 행 번호 리스트 (1-based index)
selected_rows = [75, 105, 180]

# 0-based 인덱스로 변환 (Python 인덱스는 0부터 시작)
selected_rows = [row - 1 for row in selected_rows]

# 데이터프레임에서 해당 행 번호만 필터링
filtered_df = df.iloc[selected_rows]

# 크롤링 및 저장
for idx, row in filtered_df.iterrows():
    url = row['링크']
    title = row['제목'][:30]  # 제목의 앞 30자를 파일명으로 사용

    print(f"크롤링 중 ({idx + 1}/{len(df)}): {url}")

    # 기사 내용 크롤링
    content = scrape_article_content_selenium(url)

    if content:
        try:
            # 파일명 생성
            file_name = f"Chosun_LJM_{idx + 1}_200.txt"
            file_path = os.path.join(output_dir, file_name)

            # 텍스트 파일로 저장
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)

            print(f"기사 저장 완료: {file_path}")
        except Exception as e:
            print(f"파일 저장 실패: {url}, 오류: {e}")

    else:
        print(f"본문이 비어 있습니다: {url}")


    # 요청 간 간격 두기
    time.sleep(2)

# WebDriver 종료
driver.quit()


크롤링 중 (75/200): http://monthly.chosun.com/client/news/viw.asp?ctcd=A&nNewsNumb=202203100012
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_75_200.txt
크롤링 중 (105/200): http://monthly.chosun.com/client/news/viw.asp?ctcd=A&nNewsNumb=202203100016
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_105_200.txt
크롤링 중 (180/200): http://monthly.chosun.com/client/news/viw.asp?ctcd=A&nNewsNumb=202203100014
기사 저장 완료: C:\Users\Administrator\2024-2\조선일보\이재명 기사 200\Chosun_LJM_180_200.txt
